In [76]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from typing import List, Dict
from pathlib import Path
import json

# Constantes
TIMEOUT = 10
BASE_WIKIPEDIA = "https://en.wikipedia.org"
SEARCH_URL = f"{BASE_WIKIPEDIA}/w/index.php"


In [80]:
def buscar_en_wikipedia(termino: str) -> str:
    """
    Dado un término de búsqueda, realiza una consulta en la versión en inglés de Wikipedia
    y devuelve la URL del primer resultado (o cadena vacía si no hay resultados).
    """
    params = {
        "search": termino,
        "ns0": 1  # buscar solo en artículos (namespace 0)
    }

    try:
        resp = requests.get(SEARCH_URL, params=params, timeout=TIMEOUT)
        resp.raise_for_status()
    except requests.RequestException as e:
        print(f"Error de conexión o de estado HTTP en la búsqueda: {e}")
        return ""

    soup = BeautifulSoup(resp.text, "html.parser")
    
    # 1) Intentar obtener la lista de resultados (en div.mw-search-result-heading)
    resultado = soup.select_one("div.mw-search-result-heading a")
    if not resultado:
        # Puede que no haya resultados o que Wikipedia te redirija a un artículo directamente
        link_canonical = soup.select_one("link[rel='canonical']")
        if link_canonical and link_canonical.has_attr("href"):
            return link_canonical["href"].strip()  # URL canónica del artículo
        else:
            return ""  # Sin resultados ni redirección clara

    # 2) Construir la URL con el href del primer resultado
    href = resultado.get("href", "")
    if href.startswith("/wiki/"):
        return BASE_WIKIPEDIA + href
    return ""


def obtener_primer_parrafo(url_articulo: str) -> str:
    """
    Dada la URL de un artículo de Wikipedia, devuelve el primer párrafo no vacío de su contenido.
    
    - Si la página es de desambiguación (o incluye "may refer to"), 
      busca el primer enlace <a href="/wiki/..." y visita esa página.
    - Devuelve un mensaje de error si no encuentra nada útil.
    """
    if not url_articulo:
        return "No se encontró un enlace válido al artículo."

    try:
        resp = requests.get(url_articulo, timeout=TIMEOUT)
        resp.raise_for_status()
    except requests.RequestException as e:
        return f"Error de conexión al obtener el artículo: {e}"

    # Chequear si es una página de desambiguación
    # 1) Si el texto incluye "may refer to", o
    # 2) si la URL contiene "disambiguation"
    if ("may refer to" in resp.text.lower()) or ("disambiguation" in resp.url.lower()):
        print("Página de desambiguación detectada. Intentando seguir el primer enlace relevante...")
        soup = BeautifulSoup(resp.text, "html.parser")
        # Normalmente los enlaces de desambiguación están en listas <ul><li><a>
        primer_enlace = soup.select_one("#mw-content-text ul li a[href^='/wiki/']")
        if primer_enlace:
            new_link = BASE_WIKIPEDIA + primer_enlace["href"]
            # Volvemos a llamar a la misma función para obtener el primer párrafo del nuevo enlace
            return obtener_primer_parrafo(new_link)
        else:
            return "Página de desambiguación sin enlaces válidos."

    # Si no es desambiguación, extraemos el primer párrafo
    soup = BeautifulSoup(resp.text, "html.parser")
    parrafos = soup.select("div#mw-content-text p")
    for p in parrafos:
        texto = p.get_text().strip()
        if texto:
            return texto

    return "No se encontró un párrafo con contenido en el artículo."


def descripcion_wikipedia(termino: str) -> str:
    """
    Función principal que:
    1. Busca el término en la página de resultados de Wikipedia (buscar_en_wikipedia).
    2. Obtiene la URL del primer resultado.
    3. Devuelve el primer párrafo útil de esa página (obtener_primer_parrafo),
       resolviendo automáticamente páginas de desambiguación.
    """
    url_final = buscar_en_wikipedia(termino)
    if not url_final:
        return f"No se encontraron resultados para '{termino}' en Wikipedia."

    return obtener_primer_parrafo(url_final)

In [83]:
termino_busquedas = ["super mario", "python", "kirby land", "zelda of zelda"]
for termino in termino_busquedas:
    resultado = descripcion_wikipedia(termino)
    print(f"Descripción de '{termino}':\n{resultado}\n")

Descripción de 'super mario':
Super Mario[a] (also known as Super Mario Bros.[b] and Mario)[c] is a platform game series created by Nintendo starring their mascot, Mario. It is the central series of the greater Mario franchise. At least one Super Mario game has been released for every major Nintendo video game console. However, there have also been a number of Super Mario video games released on non-Nintendo gaming platforms.[1] There are more than 20 games in the series.

Página de desambiguación detectada. Intentando seguir el primer enlace relevante...
Descripción de 'python':
The Pythonidae, commonly known as pythons, are a family of nonvenomous snakes found in Africa, Asia, and Australia. Among its members are some of the largest snakes in the world. Ten genera and 39 species are currently recognized. Being naturally non-venomous, pythons must constrict their prey to induce cardiac arrest prior to consumption. Pythons will typically strike at and bite their prey of choice to gain 

In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def buscar_en_youtube(termino):
    """
    Abre YouTube en un navegador automatizado, realiza la búsqueda del término
    y extrae los títulos de los primeros resultados.
    
    :param termino: (str) Término que queremos buscar en YouTube.
    :return: (list) Lista con los títulos de los primeros videos encontrados.
    """
    # Inicializamos el driver (en este ejemplo, Chrome)
    # Asegúrate de que el driver (chromedriver) esté instalado y en el PATH.
    driver = webdriver.Chrome()
    
    try:
        driver.get("https://www.youtube.com/")
        time.sleep(2)  # Esperamos para que cargue el sitio
        
        # Si aparece un banner de cookies, podrías cerrarlo con un .click() si es necesario
        
        # Localizamos la barra de búsqueda por NAME
        barra_busqueda = driver.find_element(By.NAME, "search_query")
        # Escribimos el término y presionamos Enter
        barra_busqueda.send_keys(termino)
        barra_busqueda.send_keys(Keys.ENTER)
        time.sleep(3)  # Esperamos a que aparezcan los resultados
        
        # Buscamos los elementos con ID "video-title"
        titulos_elementos = driver.find_elements(By.ID, "video-title")
        
        # Obtenemos el texto de cada elemento
        titulos = [titulo.text for titulo in titulos_elementos]
        
        return titulos
    finally:
        # Cerramos el navegador al final
        driver.quit()


In [49]:
# Ejemplo de uso:
termino_busqueda_youtube = "canciones de rock"  # Cambia a lo que desees buscar
resultados = buscar_en_youtube(termino_busqueda_youtube)

print(f"Resultados de la búsqueda para '{termino_busqueda_youtube}':")
for idx, titulo in enumerate(resultados, start=1):
    print(f"{idx}. {titulo}")


Resultados de la búsqueda para 'canciones de rock':
1. Nirvana, Led Zeppelin, Bon Jovi, Aerosmith, U2, ACDC 🤘 Classic Rock Songs 70s 80s 90s Full Album
2. Enrique Bunbury, Caifanes, Enanitos Verdes, Maná, Soda Estereo Rock en Espanol de los 80 y 90
3. This Heavy Metal Anthem Is Relentless
4. Metallica, Queen, Nirvana, Guns N Roses, Bon Jovi, ACDC 🔥 Best Classic Rock Songs 70s 80s 90s
5. Kiss - I Was Made For Lovin' You
6. AC/DC - Back In Black (Official 4K Video)
7. 👉 MIX ROCK en ESPAÑOL de los 80 y 90 🎵🎵🎵 CLÁSICOS DE LOS 80 & 90 Dj Suarez PUCALLPA
8. Néstor Rodríguez - Siempre Igual (Videoclip)
9. Tatiana - Navidad Rock
10. 100 CANCIONES ICONICAS DEL POP ROCK (1960-2023)
11. SALVA 1 CANCION de ROCK #2👨‍🎤🎶 🤟 ¿Cuál es tu Canción Favorita?
12. Queen - We Will Rock You (Official Video)
13. Rock para Viajar ✈️ (Soda Stereo, Enanitos Verdes, Prisioneros, Git, Virus, Vilma Palma, Fito Paez)
14. Guns N' Roses - Sweet Child O' Mine (Official Music Video)
15. Soda Stereo - De Música Ligera (Off